# Геолокация сообщений

#### Импорт библиотек

In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

import sloyka
from sloyka import Geocoder

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /home/gk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Загрузка данных

In [2]:
import pandas as pd

df = pd.read_csv('/home/gk/vscode/SOIKA_2/sloyka/sloyka/sample_data/sample_data.csv', index_col=0)

In [3]:
df.head()

,message_id,Дата и время,Текст комментария
0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...
1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...
2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя..."
3,3,2023.01.28 12:42,"2) Фурштатская, 17 Здесь прямо-таки умоляю обр..."
4,4,2023.01.28 12:45,"3) Фурштатская, 13 Отслоение штукатурного слоя..."


#### Запуск геолокатора со встроенным геокодером

In [4]:
result = Geocoder().run(df[:20], text_column='Текст комментария')

2024-03-20 22:59:14,992 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS
03-20 23:00 | INFO     | clear_names started
03-20 23:00 | INFO     | get_street started
03-20 23:00 | INFO     | extract_ner_street started


100%|██████████| 20/20 [00:00<00:00, 70.90it/s]

03-20 23:00 | INFO     | pattern1.sub started



100%|██████████| 19/19 [00:00<00:00, 10718.46it/s]

03-20 23:00 | INFO     | pattern2.findall started


...

03-20 23:00 | INFO     | extract_toponym started



100%|██████████| 19/19 [00:00<00:00, 270.98it/s]

03-20 23:00 | INFO     | get_stem started


03-20 23:00 | INFO     | create_gdf started


100%|██████████| 25/25 [00:12<00:00,  1.98it/s]


In [5]:
result

,message_id,Дата и время,Текст комментария,Street,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...,мойка,0.997,11,Мойка,улица,набережная реки Мойки 11 Санкт-Петербург Россия,['набережная реки Мойки 11 Санкт-Петербург Рос...,набережная реки Мойки 11 Санкт-Петербург Россия,набережная реки Мойки,"Business France, 11, набережная реки Мойки, Дв...",POINT (30.32437 59.94252)
1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...,рубинштейна,0.900,2,Рубинштейна,None,улица Рубинштейна 2 Санкт-Петербург Россия,['улица Рубинштейна 2 Санкт-Петербург Россия'],улица Рубинштейна 2 Санкт-Петербург Россия,улица Рубинштейна,"Parivar, 2/45, улица Рубинштейна, Владимирский...",POINT (30.34690 59.93247)
2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя...",фурштатская,0.784,19,Фурштатская,None,Фурштатская улица 19 Санкт-Петербург Россия,['Фурштатская улица 19 Санкт-Петербург Россия'],Фурштатская улица 19 Санкт-Петербург Россия,Фурштатская улица,"Цветочки, 19, Фурштатская улица, Литейный окру...",POINT (30.35583 59.94504)
3,3,2023.01.28 12:42,"2) Фурштатская, 17 Здесь прямо-таки умоляю обр...",фурштатская,0.803,17,Фурштатская,None,Фурштатская улица 17 Санкт-Петербург Россия,['Фурштатская улица 17 Санкт-Петербург Россия'],Фурштатская улица 17 Санкт-Петербург Россия,Фурштатская улица,"17, Фурштатская улица, Литейный округ, Санкт-П...",POINT (30.35526 59.94486)
4,4,2023.01.28 12:45,"3) Фурштатская, 13 Отслоение штукатурного слоя...",фурштатская,0.768,13,Фурштатская,None,Фурштатская улица 13 Санкт-Петербург Россия,['Фурштатская улица 13 Санкт-Петербург Россия'],Фурштатская улица 13 Санкт-Петербург Россия,Фурштатская улица,"13, Фурштатская улица, Литейный округ, Санкт-П...",POINT (30.35413 59.94489)
5,5,2023.01.28 12:46,"4) Фурштатская, 11 Отслоение штукатурного слоя",фурштатская,0.846,11,Фурштатская,None,Фурштатская улица 11 Санкт-Петербург Россия,['Фурштатская улица 11 Санкт-Петербург Россия'],Фурштатская улица 11 Санкт-Петербург Россия,Фурштатская улица,"Stok, 11, Фурштатская улица, Литейный округ, С...",POINT (30.35376 59.94509)
6,6,2023.01.26 10:04,Доброе утро Лиговский проспект 44 перевели на...,лиговский,0.998,44,Лиговский,проспект,Лиговский проспект 44 Санкт-Петербург Россия,['Лиговский проспект 44 Санкт-Петербург Россия'],Лиговский проспект 44 Санкт-Петербург Россия,Лиговский проспект,"Комедианты, 44, Лиговский проспект, округ Лиго...",POINT (30.35856 59.92624)
7,7,2023.01.17 16:13,[club189314823|Комитет по энергетике и инженер...,басков,0.960,17,Басков,переулок,Басков переулок 17 Санкт-Петербург Россия,['Басков переулок 17 Санкт-Петербург Россия'],Басков переулок 17 Санкт-Петербург Россия,Басков переулок,"17, Басков переулок, Литейный округ, Санкт-Пет...",POINT (30.35650 59.93932)
8,8,2023.01.19 14:30,[club189314823|Комитет по энергетике и инженер...,басков,0.996,,Басков,None,Басков переулок Санкт-Петербург Россия,['Басков переулок Санкт-Петербург Россия'],Басков переулок Санкт-Петербург Россия,Басков переулок,"Басков переулок, Литейный округ, Санкт-Петербу...",POINT (30.35778 59.93954)
9,9,2023.01.13 20:58,а метро Адмиралтейская почти постоянно стоит 2...,адмиралтейская,NaN,,Адмиралтейская,None,"Адмиралтейская улица Санкт-Петербург Россия,А...",['Адмиралтейская улица Санкт-Петербург Россия...,Адмиралтейская улица Санкт-Петербург Россия,Адмиралтейская улица,"Адмиралтейская улица, Чухонка, Колпино, Санкт-...",POINT (30.59423 59.74329)


In [6]:
pd.set_option('display.max_colwidth', 20)
pd.reset_option('display.width', 10)
# pd.set_option('display.width', 2000)


In [7]:
mask = (result.Score.isna())&(~result.Street.isna())
result[mask]
# result.to_csv('C:\Users\USER\SOIKA\сохры.csv', index=False)

,message_id,Дата и время,Текст комментария,Street,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
9,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская у...,Адмиралтейская у...,Адмиралтейская у...,POINT (30.59423 ...
10,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская у...,Адмиралтейская н...,Адмиралтейская у...,POINT (30.59423 ...
11,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская н...,Адмиралтейская у...,Адмиралтейская н...,POINT (30.30745 ...
12,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская н...,Адмиралтейская н...,Адмиралтейская н...,POINT (30.30745 ...
13,10,2023.01.13 20:41,Б. Московская-ид...,социалистическая,NaN,,Социалистическая,улица,Социалистическая...,['Социалистическ...,Социалистическая...,Социалистическая...,Социалистическая...,POINT (30.13556 ...


#### Визаулизация полученных сообщений

In [8]:
result.head(3)

,message_id,Дата и время,Текст комментария,Street,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,0,2023.01.26 16:32,Здравствуйте! В ...,мойка,0.997,11,Мойка,улица,набережная реки ...,['набережная рек...,набережная реки ...,набережная реки ...,"Business France,...",POINT (30.32437 ...
1,1,2023.01.26 11:55,[club143265175|Ц...,рубинштейна,0.900,2,Рубинштейна,None,улица Рубинштейн...,['улица Рубинште...,улица Рубинштейн...,улица Рубинштейна,"Parivar, 2/45, у...",POINT (30.34690 ...
2,2,2023.01.28 12:39,"1) Фурштатская, ...",фурштатская,0.784,19,Фурштатская,None,Фурштатская улиц...,['Фурштатская ул...,Фурштатская улиц...,Фурштатская улица,"Цветочки, 19, Фу...",POINT (30.35583 ...


In [9]:
# result.explore()